## Init methods

In [1]:
from pydantic import BaseModel, BaseSettings, root_validator, validator

class Config(BaseSettings):
    option1: str
    option2: int


class Evaluator(BaseModel):

    config: Config

    @root_validator(pre=True)
    def validate_config(cls, values):

        # check if config is instance of Config
        if values.get("config"):
            return values

        else: 
            return {"config": Config(**values)}


    

In [2]:
evaluator = Evaluator(option1="hi", option2=3)
evaluator

Evaluator(config=Config(option1='hi', option2=3))

In [3]:
config = Config(option1="hi", option2=3)
config

Config(option1='hi', option2=3)

In [4]:
evaluator_from_config = Evaluator(config=config)
evaluator_from_config

Evaluator(config=Config(option1='hi', option2=3))

# JSON encoding and decoding callable functions

In [5]:
import json
from typing import Callable
from types import FunctionType, MethodType

from pydantic import Extra

from xopt.utils import get_function 

In [6]:
JSON_ENCODERS = {
    FunctionType: lambda x: f"{x.__module__}.{x.__qualname__}",
    Callable: lambda x: f"{x.__module__}.{type(x).__qualname__}",
}

from typing import Any, Callable, Dict, Generic, Iterable, Optional, TypeVar, Tuple
ObjType = TypeVar("ObjType")
JSON_ENCODERS = {
    # function/method type distinguished for class members and not recognized as callables
    FunctionType: lambda x: f"{x.__module__}.{x.__qualname__}",
    MethodType: lambda x: f"{x.__module__}.{x.__qualname__}",
    Callable: lambda x: f"{x.__module__}.{type(x).__qualname__}",
    type: lambda x: f"{x.__module__}.{x.__name__}",
    # for encoding instances of the ObjType}
    ObjType: lambda x: f"{x.__module__}.{x.__class__.__qualname__}",
}


In [7]:

class CallableModel(BaseModel):
    callable: Callable

    class Config:
        arbitrary_types_allowed = True
        json_encoders = JSON_ENCODERS
        extra = 'forbid'

    @root_validator(pre=True)
    def validate_all(cls, values):

        callable = values.pop("callable")
        if not isinstance(
            callable,
            (
                str,
                Callable,
            ),
        ):
            raise ValueError(
                "Callable must be object or a string. Provided %s", type(callable)
            )

        values["callable"] = get_function(callable)



        return values

In [8]:
def f(x):
    return 2*x

In [9]:
m = CallableModel(callable=f)
m.callable(3)

6

In [10]:
m.json()

'{"callable": "__main__.f"}'

In [11]:
m2 = CallableModel(**json.loads(m.json()))
m2.callable(345)

690

In [12]:
def f():
    pass

# or this
f = lambda x: 2*x

type(f) is FunctionType

True

# Evaluator

In [17]:
import json
from typing import Callable
from types import FunctionType, MethodType

from pydantic import Extra, Field

from xopt.pydantic import NormalExecutor
from xopt.utils import get_function, get_function_defaults

In [21]:
from concurrent.futures import ProcessPoolExecutor
from xopt.evaluator import DummyExecutor

JSON_ENCODERS = {
    FunctionType: lambda x: f"{x.__module__}.{x.__qualname__}",
    Callable: lambda x: f"{x.__module__}.{type(x).__qualname__}",
}

class Evaluator(BaseModel):
    function: Callable
    max_workers: int = 1
    executor: NormalExecutor = Field(exclude=True)
    function_kwargs: dict = {}
    

    class Config:
        arbitrary_types_allowed = True
        # validate_assignment = True # Broken in 1.9.0. Trying to fix in https://github.com/samuelcolvin/pydantic/pull/4194
        json_encoders = JSON_ENCODERS
        extra = 'forbid'

    @root_validator(pre=True)
    def validate_all(cls, values):
   
        f = get_function(values["function"])
        kwargs = values.get("function_kwargs", {})
        kwargs = {**get_function_defaults(f), **kwargs}
        values["function"] = f
        values["function_kwargs"] = kwargs

        max_workers = values.pop("max_workers", 1)

        executor = values.pop("executor", None)
        if not executor:
            if max_workers > 1:
                executor = ProcessPoolExecutor(max_workers=max_workers)
            else: 
                executor = DummyExecutor()

        # Cast as a NormalExecutor
        values["executor"] =  NormalExecutor[type(executor)](executor=executor)
        values["max_workers"] = max_workers
        
        return values    



def g(a, b=2):
    return a*b

ev = Evaluator(function=g, function_kwargs={'b':3}, max_workers=2, executor=None)
ev.executor = None
ev.json()

'{"function": "__main__.g", "max_workers": 2, "function_kwargs": {"b": 3}}'

root validator {'function': <function g at 0x15bb2a4c0>, 'function_kwargs': {'b': 3}, 'max_workers': 2, 'executor': None}
done loader=ObjLoader[ProcessPoolExecutor](object=None, loader=CallableModel(callable=<class 'concurrent.futures.process.ProcessPoolExecutor'>, kwargs=Kwargs_ProcessPoolExecutor(max_workers=None, mp_context=None, initializer=None, initargs=None)), object_type=<class 'concurrent.futures.process.ProcessPoolExecutor'>) executor_type=<class 'concurrent.futures.process.ProcessPoolExecutor'> submit_callable='submit' map_callable='map' shutdown_callable='shutdown' executor=<concurrent.futures.process.ProcessPoolExecutor object at 0x15bb277c0>
root validator {'function': <function g at 0x15bb2a4c0>, 'max_workers': 2, 'executor': None, 'function_kwargs': {'b': 3}}
done loader=ObjLoader[ProcessPoolExecutor](object=None, loader=CallableModel(callable=<class 'concurrent.futures.process.ProcessPoolExecutor'>, kwargs=Kwargs_ProcessPoolExecutor(max_workers=None, mp_context=None, i

ValidationError: 1 validation error for Evaluator
executor
  none is not an allowed value (type=type_error.none.not_allowed)

In [240]:
ev.executor 

NormalExecutor[ProcessPoolExecutor](loader=ObjLoader[ProcessPoolExecutor](object=None, loader=CallableModel(callable=<class 'concurrent.futures.process.ProcessPoolExecutor'>, kwargs=Kwargs_ProcessPoolExecutor(max_workers=None, mp_context=None, initializer=None, initargs=None)), object_type=<class 'concurrent.futures.process.ProcessPoolExecutor'>), executor_type=<class 'concurrent.futures.process.ProcessPoolExecutor'>, submit_callable='submit', map_callable='map', shutdown_callable='shutdown', executor=<concurrent.futures.process.ProcessPoolExecutor object at 0x15b853b50>)

In [228]:
ev.max_workers = 1
ev.executor=None

root validator {'function': <function g at 0x15b8c85e0>, 'max_workers': 1, 'executor': NormalExecutor[ProcessPoolExecutor](loader=ObjLoader[ProcessPoolExecutor](object=None, loader=CallableModel(callable=<class 'concurrent.futures.process.ProcessPoolExecutor'>, kwargs=Kwargs_ProcessPoolExecutor(max_workers=None, mp_context=None, initializer=None, initargs=None)), object_type=<class 'concurrent.futures.process.ProcessPoolExecutor'>), executor_type=<class 'concurrent.futures.process.ProcessPoolExecutor'>, submit_callable='submit', map_callable='map', shutdown_callable='shutdown', executor=<concurrent.futures.process.ProcessPoolExecutor object at 0x15b8e43d0>), 'function_kwargs': {'b': 3}}
root validator {'function': <function g at 0x15b8c85e0>, 'function_kwargs': {'b': 3}, 'executor': None, 'max_workers': 1}


ValidationError: 1 validation error for Evaluator
executor
  none is not an allowed value (type=type_error.none.not_allowed)

In [17]:
ev.json()

'{"xfunction": "__main__.g", "max_workers": 1, "function_kwargs": {"b": 3}}'

# Executors

In [18]:
from concurrent.futures import ThreadPoolExecutor

In [19]:
with ThreadPoolExecutor() as executor:
    #print(dir(executor))
    print(type(executor))

<class 'concurrent.futures.thread.ThreadPoolExecutor'>


In [20]:
from xopt.pydantic import NormalExecutor

In [99]:
class ExecutorModel(BaseModel):
    executor: NormalExecutor

    class Config:
        arbitrary_types_allowed = True
        #json_encoders = {NormalExecutor: 'dummy'}
        extra = 'forbid'

    @root_validator(pre=True)
    def validate_all(cls, values):
        executor = values.pop("executor")
        values["executor"] =  NormalExecutor[type(executor)](executor=executor)
        print(values)
        return values

executor = ThreadPoolExecutor()
ne = NormalExecutor[type(executor)](executor=executor)
ne.json()

'{"loader": {"object": null, "loader": {"callable": "concurrent.futures.thread.ThreadPoolExecutor", "kwargs": {"max_workers": null, "initializer": null, "initargs": null, "thread_name_prefix": ""}}, "object_type": "concurrent.futures.thread.ThreadPoolExecutor"}, "submit_callable": "submit", "map_callable": "map", "shutdown_callable": "shutdown", "executor": "concurrent.futures.thread.ThreadPoolExecutor"}'

In [97]:
type(executor)

concurrent.futures.thread.ThreadPoolExecutor

In [95]:
from xopt.pydantic import BaseExecutor, ObjType
from pydantic import Field

class Evaluator(BaseModel):
    executor: BaseExecutor = Field(exclude=True)

executor = ThreadPoolExecutor()
ev = Evaluator(executor=executor)
ev.json()


ValidationError: 1 validation error for Evaluator
executor
  value is not a valid dict (type=type_error.dict)

In [92]:
from xopt.pydantic import BaseExecutor, ObjType
from pydantic import Field

class Evaluator(BaseModel):
    executor: BaseExecutor = Field(exclude=True)

    @root_validator(pre=True)
    def validate_all(cls, values):
        print(values)
        if values.get("executor"):
            executor_val = values.get("executor")
            print('here 1')
            if isinstance(executor_val, (BaseExecutor,)):
                print('here 2')
                pass

            elif isinstance(executor_val, (str,)):
                print('here 3')
                #executor_type = get_executor_type(executor_val)

                # prob would have to handle instantiation values
                #values["executor"] = executor_type()

            elif isinstance(executor_val, (type,)):
                # also handle args, kwargs...
                print('here 4')
                values["executor"] = BaseExecutor(executor=executor_val)
            else:
                print('here 5')
                values["executor"] = BaseExecutor(executor=executor_val)
        return values

executor = ThreadPoolExecutor()       
#Evaluator(executor=executor) 

custom_executor = NormalExecutor[type(executor)](executor=executor)

custom_executor.submit


<bound method NormalExecutor.submit of NormalExecutor[ThreadPoolExecutor](loader=ObjLoader[ThreadPoolExecutor](object=None, loader=CallableModel(callable=<class 'concurrent.futures.thread.ThreadPoolExecutor'>, kwargs=Kwargs_ThreadPoolExecutor(max_workers=None, initializer=None, initargs=None, thread_name_prefix='')), object_type=<class 'concurrent.futures.thread.ThreadPoolExecutor'>), executor_type=<class 'concurrent.futures.thread.ThreadPoolExecutor'>, submit_callable='submit', map_callable='map', shutdown_callable='shutdown', executor=<concurrent.futures.thread.ThreadPoolExecutor object at 0x15b3fe280>)>

In [39]:
def f(x):
    return 2*x

inputs = [1,2,3]

with ThreadPoolExecutor() as executor:
    results = list(executor.map(f, inputs))
    print(results)

[2, 4, 6]


In [41]:
def f(x):
    return 2*x

inputs = [1,2,3]


with ThreadPoolExecutor() as executor:
    print(executor)
    ex = ExecutorModel(executor=executor)
    #print(dir(ex.executor))
    #print(ex.json())

    print(ex.executor)
   
    results = list(map(f, inputs))

    ex.executor.map(f, inputs)

    print(results)

    print(ex.json())

loader=ObjLoader[ThreadPoolExecutor](object=None, loader=CallableModel(callable=<class 'concurrent.futures.thread.ThreadPoolExecutor'>, kwargs=Kwargs_ThreadPoolExecutor(max_workers=None, initializer=None, initargs=None, thread_name_prefix='')), object_type=<class 'concurrent.futures.thread.ThreadPoolExecutor'>) executor_type=<class 'concurrent.futures.thread.ThreadPoolExecutor'> submit_callable='submit' map_callable='map' shutdown_callable='shutdown' executor=<concurrent.futures.thread.ThreadPoolExecutor object at 0x1584a5520>
[2, 4, 6]


TypeError: Object of type 'type' is not JSON serializable

In [24]:
list(map(f,[1, 2, 3, 4]) )

[2, 4, 6, 8]

In [34]:
def f2(x, y):
    print(x, y)

with ThreadPoolExecutor(max_workers=1) as executor:
    r = list(executor.map(f2, [1,2], [3,4]) )
    print(r)

1 3
2 4
[None, None]


In [164]:
from typing import List
from pydantic import BaseModel, ValidationError, validator


class DemoModel(BaseModel):
    square_numbers: List[int] = []
    cube_numbers: List[int] = []

    # '*' is the same as 'cube_numbers', 'square_numbers' here:
    @validator('*', pre=True)
    def split_str(cls, v):
        if isinstance(v, str):
            return v.split('|')
        return v

    @validator('cube_numbers', 'square_numbers')
    def check_sum(cls, v):
        if sum(v) > 42:
            raise ValueError('sum of numbers greater than 42')
        return v

    @validator('square_numbers', each_item=True)
    def check_squares(cls, v):
        assert v ** 0.5 % 1 == 0, f'{v} is not a square number'
        return v

    @validator('cube_numbers', each_item=True)
    def check_cubes(cls, v):
        # 64 ** (1 / 3) == 3.9999999999999996 (!)
        # this is not a good way of checking cubes
        assert v ** (1 / 3) % 1 == 0, f'{v} is not a cubed number'
        return v

m = DemoModel(square_numbers=[1, 4, 43])        

m.cube_numbers = 43

ValidationError: 1 validation error for DemoModel
square_numbers -> 2
  43 is not a square number (type=assertion_error)

In [276]:
from pydantic import BaseModel, validator, root_validator

class M(BaseModel):
    x : int
    y: int

    class Config:
        validate_assignment = True

    @root_validator(pre=True)
    def xxx(cls, values):
        print('root validator', values)
        values["x"] = "a"
        return values

    @validator("x", pre=True)
    def x_setter(cls, v):
        print('x_setter', v)
        v = 666
        return v


m = M(x='b', y=99)
m.x = Exception
m

root validator {'x': 'b', 'y': 99}
x_setter a
root validator {'x': <class 'Exception'>, 'y': 99}
x_setter <class 'Exception'>


M(x=666, y=99)

In [214]:
import numpy as np 

#x = np.nan
x = np.inf

x > 0, x < 0 

(True, False)

In [216]:
np.inf * -1

-inf

In [1]:
from pydantic import BaseModel, root_validator

class M(BaseModel):

    x: int = 1

    class Config:
        validate_assignment = True

    @root_validator(pre=True)
    def validate_all(cls, values):
        print('root validator', values)
        values["x"] = 99  # Always hijack x
        print('done', values)
        return values    


m = M(x=3)
m.x = None  # ValidationError: 1 validation error for M

root validator {'x': 3}
done {'x': 99}
root validator {'x': None}
done {'x': 99}


ValidationError: 1 validation error for M
x
  none is not an allowed value (type=type_error.none.not_allowed)

In [2]:
import pydantic

In [3]:
pydantic.__version__

'1.9.0'